# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,26.21,76,22,4.76,MU,1702176286
1,1,talnakh,69.4865,88.3972,-34.38,99,100,3.45,RU,1702176316
2,2,san antonio de pale,-1.4014,5.6325,27.16,80,51,5.23,GQ,1702176341
3,3,ust-nera,64.5667,143.2000,-16.26,99,100,1.88,RU,1702176319
4,4,new norfolk,-42.7826,147.0587,21.51,24,8,1.55,AU,1702176402


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity",
scale = .8,
color = "City",
tiles = "OSM",
alpha=0.5)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']<=27) #Max temp less than or equal to 27
                            &(city_data_df['Max Temp']>=14)     #Max temp greater than or equal to 14
                            &(city_data_df['Cloudiness']<=50)   #Cloudiness less than or equal to 50
                            &(city_data_df['Humidity']<55),:]   #Humidity less than 55


# Drop any rows with null values
ideal_city_df.dropna()
city_count = len(ideal_city_df)

# Display sample data
print (f'Total ideal cities: {city_count}')
ideal_city_df.head()

Total ideal cities: 33


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,new norfolk,-42.7826,147.0587,21.51,24,8,1.55,AU,1702176402
8,8,dedougou,12.4667,-3.4667,21.52,16,49,3.28,BF,1702176560
18,18,margaret river,-33.9500,115.0667,23.76,29,0,0.34,AU,1702176291
56,56,duba,27.3513,35.6901,21.85,41,38,1.62,SA,1702176490
61,61,blackmans bay,-43.0167,147.3167,20.85,49,0,0.45,AU,1702176290


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,new norfolk,AU,-42.7826,147.0587,24,
8,dedougou,BF,12.4667,-3.4667,16,
18,margaret river,AU,-33.9500,115.0667,29,
56,duba,SA,27.3513,35.6901,41,
61,blackmans bay,AU,-43.0167,147.3167,49,
89,chak two hundred forty-nine thal development a...,PK,31.1796,71.2095,34,
121,collie,AU,-33.3667,116.1500,17,
143,tchintabaraden,NE,15.8969,5.7985,17,
171,guerrero negro,MX,27.9769,-114.0611,43,
210,kandi,BJ,11.1342,2.9386,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
          'apiKey':geoapify_key,
          'limit':1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
new norfolk - nearest hotel: Woodbridge on the Derwent
dedougou - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
duba - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
chak two hundred forty-nine thal development authority - nearest hotel: No hotel found
collie - nearest hotel: Federal Hotel
tchintabaraden - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
kandi - nearest hotel: God Blessing
diego de almagro - nearest hotel: Residencial Hambyll
mendoza - nearest hotel: Hotel Eben-Ezer
nawabshah - nearest hotel: No hotel found
huotong - nearest hotel: No hotel found
tumayr - nearest hotel: No hotel found
bentiu - nearest hotel: Peace Hotel
gwadar - nearest hotel: Sadaf Resorts
tchaourou - nearest hotel: Auberge
moundou - nearest hotel: Hôtel Djimra
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
hasaki - nearest hotel: 大新旅館
bara uchana - nearest hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
4,new norfolk,AU,-42.7826,147.0587,24,Woodbridge on the Derwent
8,dedougou,BF,12.4667,-3.4667,16,No hotel found
18,margaret river,AU,-33.9500,115.0667,29,Margaret River Hotel
56,duba,SA,27.3513,35.6901,41,No hotel found
61,blackmans bay,AU,-43.0167,147.3167,49,Villa Howden
89,chak two hundred forty-nine thal development a...,PK,31.1796,71.2095,34,No hotel found
121,collie,AU,-33.3667,116.1500,17,Federal Hotel
143,tchintabaraden,NE,15.8969,5.7985,17,No hotel found
171,guerrero negro,MX,27.9769,-114.0611,43,Plaza sal paraiso
210,kandi,BJ,11.1342,2.9386,14,God Blessing


In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
color = "City",
tiles = "OSM",
hover_cols = (["Hotel Name","Country"]),
scale = 2,
size= "Humidity",
alpha=.8)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)